# Advanced Multi-Agent Reasoning with AutoGen

In this notebook, we'll explore how to build multi-agent systems with enhanced reasoning capabilities using AutoGen. We'll extend the concepts from previous notebooks by adding a dedicated reasoning agent that can evaluate conversations and improve outcomes.

## What is Reasoning in Multi-Agent Systems?

Reasoning in multi-agent systems involves:
- Evaluating the quality and consistency of agent interactions
- Identifying contradictions or logical errors in conversations
- Providing meta-feedback to improve the overall system performance
- Using specialized models (like o1-mini) that excel at logical reasoning tasks

Let's start by setting up our environment.

In [ ]:
# Install required packages if needed
# !pip install autogen-agentchat autogen-ext python-dotenv pytz

In [ ]:
# Import necessary libraries
import asyncio
import os
import pytz
from datetime import datetime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams._group_chat._magentic_one._magentic_one_orchestrator import MagenticOneOrchestrator
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient, AzureOpenAIChatCompletionClient
from autogen_core import CancellationToken
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

## 1. Setting Up Azure OpenAI with Multiple Models

For advanced reasoning, we'll use multiple LLM models - a standard model for general tasks and a specialized reasoning model (o1-mini) optimized for logical reasoning tasks.

In [ ]:
if os.environ.get("GITHUB_TOKEN") is None:
    model_client = AzureOpenAIChatCompletionClient(
        azure_deployment=os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
        model=os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
        model_info={
            "json_output": True,
            "function_calling": True,
            "vision": False,
            "family": "unknown",
        },
    )

    # Update this to o1 model when available
    o1_model_client = AzureOpenAIChatCompletionClient(
        azure_deployment=os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
        model=os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
        model_info={
            "json_output": True,
            "function_calling": True,
            "vision": False,
            "family": "unknown",
        },
    )
else:
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings. 
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    token = os.environ["GITHUB_TOKEN"]
    endpoint = "https://models.inference.ai.azure.com"

    # Standard model for general tasks
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o-mini",
        base_url=endpoint,
        api_key=token
    )

    # Specialized model for reasoning tasks
    o1_model_client = OpenAIChatCompletionClient(
        model="o1-mini",
        base_url=endpoint,
        api_key=token
    )

## 2. Creating a Dedicated Reasoning Agent

We'll create a dedicated reasoning agent that can analyze conversations for inconsistencies, open questions, and contradictions. This agent uses the o1-mini model, which is specifically designed for logical reasoning tasks.

In [ ]:
# Create a reasoning agent with the o1-mini model
reasoning_agent = AssistantAgent(
    name="reasoning_agent", 
    model_client=o1_model_client, 
    system_message=None,
    description="A helpful assistant that can check the quality of the conversation and provide feedback to the agents. The checker agent can provide feedback on the quality of the conversation, the relevance of the responses, and the overall satisfaction of the user. The checker agent can also provide suggestions for improvement to the agents and should be consulted before completing the last task.",
    tools=None
)

# Define a conversation checking tool that uses the reasoning agent
async def check_conversation(messages: str) -> str:
    print("executing check_conversation")
    response = await reasoning_agent.on_messages(
                [TextMessage(content=f"Check the following messages for inconsistencies, open questions and contradictions and give concrete feedback. Here are some facts that might help: Dennis lives in somewhere in Germany. Messages: {messages}", source="user")], CancellationToken()
                )
    print(response)
    return f"This is my feedback {response}."

## 3. Creating Domain-Specific Tools

Now let's define various tools that our specialized agents will use:

In [ ]:
# Define tools for our agents
async def get_weather(city: str) -> str:
    print("executing get_weather")
    return f"The weather in {city} is 73 degrees and Sunny."

async def get_medical_history(username: str) -> str:
    """Get the medical history for a given username with known allergies and food restrictions."""
    print("executing get_medical_history")
    return f"{username} has an allergy to peanuts and eggs."

async def get_available_incredients(location: str) -> str:
    """Get the available incredients for a given location."""
    print("executing get_available_incredients")
    return f"Available incredients in {location} are: eggs, milk, bread, peanuts, beer, wine, salmon, spinache, oil and butter."

def get_current_username(input: str) -> str:
    """Get the username of the current user."""
    print("executing get_current_username")
    return "Dennis"

def get_current_location_of_user(username: str) -> str:
    """Get the current timezone location of the user for a given username."""
    print("executing get_current_location")
    print(username)
    if "Dennis" in username:
        return "Europe/Berlin"
    else:
        return "America/New_York"

def get_current_time(location: str) -> str:
    """Get the current time in the given location. The pytz is used to get the timezone for that location. Location names should be in a format like America/Seattle, Asia/Bangkok, Europe/London. Anything in Germany should be Europe/Berlin"""
    try:
        print("get current time for location: ", location)
        timezone = pytz.timezone(location)
        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")
        return current_time
    except Exception as e:
        print("Error: ", e)
        return "Sorry, I couldn't find the timezone for that location."

# Test one of our tools
await get_weather("Berlin")

## 4. Creating Specialized Agents

Now we'll create specialized agents, each with specific tools and responsibilities:

In [ ]:
# User information agent
users_agent = AssistantAgent(
    "users_agent",
    model_client=model_client,
    tools=[get_current_username, get_medical_history],
    description="A helpful assistant that can knows things about the user like the username.",
    system_message="You are a helpful assistant that can retrieve the username of the current user.",
)

# Location agent
location_agent = AssistantAgent(
    "location_agent",
    model_client=model_client,
    tools=[get_current_location_of_user],
    description="A assistant that can find the physical location of a user.",
    system_message="You are a helpful assistant that can suggest details for a location and can utilize any context information provided.",
)

# Time agent
time_agent = AssistantAgent(
    "time_agent",
    model_client=model_client,
    tools=[get_current_time],
    description="A helpful assistant that knows time in a specific location.",
    system_message="You are a helpful assistant that can retrieve the current time for a given location.",
)

# Chef agent
chef_agent = AssistantAgent(
    "chef_agent",
    model_client=model_client,
    tools=[get_available_incredients],
    description="A helpful assistant that can suggest meals and dishes for the right time of the day, location, available ingredients, user preferences and allergies.",
    system_message="You are a helpful assistant that can recommend dishes for the right time of the day, location, available ingredients and user preferences. Make sure you ask for individual food preferences and allergies as input.",
)

## 5. Creating Synthesis and Consultation Agents

In addition to our specialized agents, we need agents that can synthesize information and provide feedback:

In [ ]:
# Summary agent for synthesizing information - not used in chat
summary_agent = AssistantAgent(
    "summary_agent",
    model_client=model_client,
    description="A helpful assistant that can summarize details about conversations.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and leverage them to answer questions. You must ensure that you use that the other agents can solve the problem. When all open questions have been answered, you can respond with TERMINATE.",
)

# Consultation agent for checking conversation quality
consultation_agent = AssistantAgent(
    name="consultation_agent", 
    model_client=model_client, 
    system_message="Your task is to check the complete message flow for inconsistencies, open questions and contradictions and give concrete feedback. You should also provide suggestions for improvement to the agents and should be consulted before completing the last task.",
    description="A helpful assistant that can check the quality of the conversation and provide feedback to the agents. The checker agent can provide feedback on the quality of the conversation, the relevance of the responses, and the overall satisfaction of the user. The checker agent can also provide suggestions for improvement to the agents and should be consulted before completing the last task.",
    tools=[check_conversation]
)

## 6. Running a Multi-Agent System with Reasoning

Now we'll run our multi-agent system with the reasoning capability integrated:

In [ ]:
async def run_reasoning_multi_agent():
    # Set a message limit to prevent infinite conversations
    inner_termination = MaxMessageTermination(20)
    
    # Create a MagenticOne group chat with our agents
    magenticteam = MagenticOneGroupChat(
        [users_agent, location_agent, time_agent, chef_agent, consultation_agent], 
        model_client=model_client, 
        termination_condition=inner_termination
    )

    # Run the team and stream messages to the console
    stream = magenticteam.run_stream(task="I want to have something to eat. What would you recommend?")
    await Console(stream)

# Run our multi-agent system
await run_reasoning_multi_agent()

## 7. Understanding the Role of the Reasoning Agent

The reasoning agent plays a critical role in our multi-agent system by:

1. **Consistency Checking**: Identifying contradictions in the conversation.
2. **Gap Detection**: Finding unanswered questions or missing information.
3. **Logical Analysis**: Ensuring that conclusions follow logically from premises.
4. **Quality Improvement**: Providing feedback to other agents to improve their responses.

The o1-mini model is particularly well-suited for this task as it's designed for logical reasoning and analysis.

## 8. Expanding the System - Exercise

Try modifying the system to enhance its reasoning capabilities:

1. Add a fact-checking agent that verifies statements made by other agents.
2. Implement a planning agent that can structure the conversation to reach goals more efficiently.
3. Create a contradiction resolution mechanism when the reasoning agent identifies logical inconsistencies.

Here's a template to get you started:

In [ ]:
# Create a fact-checking agent
# fact_checking_agent = AssistantAgent(
#     "fact_checker",
#     model_client=o1_model_client,  # Using the reasoning-focused model
#     description="An agent that verifies factual claims made during the conversation.",
#     system_message="You are responsible for checking factual claims made by other agents. When you spot a claim that needs verification, research it and provide accurate information.",
# )

# Define a function to analyze contradictions
# async def resolve_contradiction(claim1: str, claim2: str) -> str:
#     """Analyzes two contradicting claims and attempts to resolve the contradiction."""
#     # Your implementation here
#     return "Resolution explanation"

## 9. Conclusion

In this notebook, we've explored how to enhance multi-agent systems with advanced reasoning capabilities:

1. We created a dedicated reasoning agent using a specialized model (o1-mini).
2. We implemented a consultation mechanism to check conversation quality.
3. We integrated this reasoning capability into a collaborative multi-agent system.
4. We explored how reasoning enhances the overall performance of agent conversations.

By adding meta-cognitive abilities to our multi-agent systems, we can create more robust, accurate, and logically sound AI assistants capable of handling complex tasks while maintaining internal consistency.

For more advanced applications, consider:
- Incorporating multiple reasoning agents with different specialties
- Adding formal logic verification
- Implementing self-correction mechanisms based on reasoning feedback

These techniques help move AI systems from simple pattern matching to more sophisticated reasoning-based intelligence.